# NanoEventsFactory example

Load a single file from x-cache (using redirector_ucsd) or the grid (using redirector_fnal), and play with the events.


In [1]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import Weights, PackedSelection

from Tools.config_helpers import redirector_fnal, redirector_ucsd
from Tools.nano_mapping import make_fileset
from Tools.helpers import get_samples

import awkward as ak

samples = get_samples()

fileset = make_fileset(['top'], samples, redirector=redirector_ucsd, small=True)

# load a subset of events
n_max = 5000
events = NanoEventsFactory.from_root(
    fileset[list(fileset.keys())[0]][0],
    schemaclass = NanoAODSchema,
    entry_stop = n_max).events()



In [2]:
events.event

<Array [4590003, 4590004, ... 4594997, 4594998] type='5000 * uint64[parameters={...'>

In [3]:
ev = events[events.event==4590003]

In [4]:
ak.to_pandas(ev.MET)

,MetUnclustEnUpDeltaX,MetUnclustEnUpDeltaY,covXX,covXY,covYY,fiducialGenPhi,fiducialGenPt,phi,pt,significance,sumEt,sumPtUnclustered
entry,,,,,,,,,,,,
0,-3.009766,4.761719,411.0,-37.5,404.0,2.337891,65.1875,1.943115,51.547745,6.210938,1102.0,940.0


In [5]:
import time

from Tools.objects import Collections

start_time = time.time()
electrons = Collections(events, 'Electron', 'tight', verbose=1).get()
muons = Collections(events, 'Muon', 'tight', verbose=1).get()


delta_time = time.time()-start_time

print ("\nTook %s seconds"%delta_time)


## Electron selection for WP tight ##
 - pt >= 15
 - abs(etaSC) <= 2.5
 - convVeto == 1
 - lostHits == 0
 - tightCharge == 2
 - abs(dxy) <= 0.05
 - abs(dz) <= 0.1
 - abs(sip3d) <= 4.0
 - tight electron MVA ID
 - custom multi isolation
 - trigger safe
 - custom ID and multi-isolation
Found 1784 objects passing the selection

## Muon selection for WP tight ##
 - pt >= 15
 - abs(eta) <= 2.4
 - mediumId == 1
 - looseId == 1
 - abs(dxy) <= 0.05
 - abs(dz) <= 0.05
 - abs(sip3d) <= 4
 - ptErrRel <= 0.2
 - custom multi isolation
 - custom multi-isolation
Found 2622 objects passing the selection

Took 1.2086637020111084 seconds


In [6]:
events[ak.num(muons, axis=1)>0].event

<Array [4590009, 4590007, ... 4594989, 4594990] type='2290 * uint64[parameters={...'>

In [7]:
muons.fields

['charge',
 'cleanmask',
 'dxy',
 'dxyErr',
 'dxybs',
 'dz',
 'dzErr',
 'eta',
 'fsrPhotonIdx',
 'fsrPhotonIdxG',
 'genPartFlav',
 'genPartIdx',
 'genPartIdxG',
 'highPtId',
 'highPurity',
 'inTimeMuon',
 'ip3d',
 'isGlobal',
 'isPFcand',
 'isTracker',
 'jetIdx',
 'jetIdxG',
 'jetPtRelv2',
 'jetRelIso',
 'looseId',
 'mass',
 'mediumId',
 'mediumPromptId',
 'miniIsoId',
 'miniPFRelIso_all',
 'miniPFRelIso_chg',
 'multiIsoId',
 'mvaId',
 'mvaLowPt',
 'mvaTTH',
 'nStations',
 'nTrackerLayers',
 'pdgId',
 'pfIsoId',
 'pfRelIso03_all',
 'pfRelIso03_chg',
 'pfRelIso04_all',
 'phi',
 'pt',
 'ptErr',
 'segmentComp',
 'sip3d',
 'softId',
 'softMva',
 'softMvaId',
 'tightCharge',
 'tightId',
 'tkIsoId',
 'tkRelIso',
 'triggerIdLoose',
 'tunepRelPt',
 'absMiniIso',
 'ptErrRel',
 'deepJet',
 'jetRelIsoV2',
 'conePt']

In [8]:
fields_to_show = ["pt", "conePt", "eta", "phi", "miniPFRelIso_all", "deepJet"]

ak.to_pandas(muons[events.event==4590009][fields_to_show])

,,pt,conePt,eta,phi,miniPFRelIso_all,deepJet
entry,subentry,,,,,,
0,0,89.001289,91.74375,-1.041992,1.666748,0.0,0.006035


In [9]:
ak.to_pandas(muons[ak.num(muons)>0][fields_to_show])

,,pt,conePt,eta,phi,miniPFRelIso_all,deepJet
entry,subentry,,,,,,
0,0,89.001289,91.743750,-1.041992,1.666748,0.000000,0.006035
1,0,22.520767,23.400000,-0.732056,-2.039551,0.031200,0.014496
2,0,35.408287,38.896875,1.223389,2.832031,0.000000,0.022186
3,0,39.832733,42.187500,-1.335205,-1.277832,0.007453,0.005619
4,0,97.141785,99.843750,-0.975586,-2.340820,0.000000,0.006428
...,...,...,...,...,...,...,...
2285,0,34.735882,35.718750,1.778320,3.064941,0.000000,0.008408
2286,0,81.104309,81.000000,0.807251,0.137268,0.000000,0.004791
2287,0,25.945396,25.101562,-0.316895,0.740234,0.000000,0.013512


You can do anything that you're usually doing inside the processor here as well, so this is very useful for debugging

In [10]:
matched_electrons = electrons[electrons.genPartIdx>=0]
sum(ak.num(matched_electrons, axis=1))

1776

In [11]:
f_in = '/store/mc/RunIIAutumn18NanoAODv7/QCD_Pt-120to170_MuEnrichedPt5_TuneCP5_13TeV_pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/70000/DE335891-829A-B943-99BE-E5A179F5F3EB.root'

events = NanoEventsFactory.from_root(
    redirector_ucsd + f_in,
    schemaclass = NanoAODSchema,
    entry_stop = 9999999).events()

muons = Collections(events, 'Muon', 'tight', verbose=1).get()

single_mu_ev = events[ak.num(muons)>0]

event_list_tight = single_mu_ev.event
event_list_tight


## Muon selection for WP tight ##
 - pt >= 15
 - abs(eta) <= 2.4
 - mediumId == 1
 - looseId == 1
 - abs(dxy) <= 0.05
 - abs(dz) <= 0.05
 - abs(sip3d) <= 4
 - ptErrRel <= 0.2
 - custom multi isolation
 - custom multi-isolation
Found 652 objects passing the selection


<Array [29552, 33557, ... 12024433, 12035747] type='651 * uint64[parameters={"__...'>

In [12]:
import numpy as np
event_selector = (np.zeros_like(events.MET.pt) == 1)

In [13]:
my_events = [29552, 12024433]

In [14]:
for ev in my_events:
    event_selector = (event_selector | (events.event == ev))

In [28]:
mu_df = ak.to_pandas(ak.flatten(muons[event_selector][fields_to_show]))
mu_df

,pt,conePt,eta,phi,miniPFRelIso_all,deepJet
entry,,,,,,
0,22.196507,73.3500,-0.518066,-0.862671,0.101121,0.974121
1,29.695017,93.2625,-1.611572,-1.919434,0.100588,0.124390


In [35]:
ev_df = ak.to_pandas(events.MET[event_selector])
ev_df

,MetUnclustEnUpDeltaX,MetUnclustEnUpDeltaY,covXX,covXY,covYY,fiducialGenPhi,fiducialGenPt,phi,pt,significance,sumEt,sumPtUnclustered
entry,,,,,,,,,,,,
0,-7.730469,-7.636719,860.0,-62.375000,1500.0,-1.039062,9.304688,0.413147,49.778206,2.777344,2454.0,1626.0
1,-5.054688,-2.496094,514.0,-5.484375,390.0,-1.850586,20.265625,-1.706299,52.056957,6.945312,1174.0,1016.0


In [36]:
import pandas as pd
pd.concat([mu_df, ev_df], axis=1, )

,pt,conePt,eta,phi,miniPFRelIso_all,deepJet,MetUnclustEnUpDeltaX,MetUnclustEnUpDeltaY,covXX,covXY,covYY,fiducialGenPhi,fiducialGenPt,phi,pt,significance,sumEt,sumPtUnclustered
entry,,,,,,,,,,,,,,,,,,
0,22.196507,73.3500,-0.518066,-0.862671,0.101121,0.974121,-7.730469,-7.636719,860.0,-62.375000,1500.0,-1.039062,9.304688,0.413147,49.778206,2.777344,2454.0,1626.0
1,29.695017,93.2625,-1.611572,-1.919434,0.100588,0.124390,-5.054688,-2.496094,514.0,-5.484375,390.0,-1.850586,20.265625,-1.706299,52.056957,6.945312,1174.0,1016.0
